In [1]:
# import os
# import random
# import shutil

# # 原始数据集根文件夹
# dataset_root = '/local/data1/honzh073/data/hospital_43'

# # 目标数据集文件夹
# target_dataset_folder = '/local/data1/honzh073/data/hospital_43_5fold'

# # 创建目标数据集文件夹
# os.makedirs(target_dataset_folder, exist_ok=True)

# # AFF和NFF文件夹
# fracture_types = ['AFF', 'NFF']

# # 分割数据集的比例
# train_ratio = 0.7
# test_ratio = 0.3

# # 遍历AFF和NFF文件夹
# for fracture_type in fracture_types:
#     source_folder = os.path.join(dataset_root, fracture_type)
#     patients = os.listdir(source_folder)
    
#     # 随机打乱患者顺序
#     random.shuffle(patients)
    
#     # 划分数据集
#     total_patients = len(patients)
#     train_end = int(total_patients * train_ratio)
    
#     train_patients = patients[:train_end]
#     # val_patients = patients[train_end:val_end]
#     test_patients = patients[train_end:]
    
#     # 复制图片到相应的数据集文件夹
#     for patient_id in train_patients:
#         source_path = os.path.join(source_folder, patient_id)
#         target_path = os.path.join(target_dataset_folder, 'train', fracture_type, patient_id)
#         shutil.copytree(source_path, target_path)
    
#     # for patient_id in val_patients:
#     #     source_path = os.path.join(source_folder, patient_id)
#     #     target_path = os.path.join(target_dataset_folder, 'val', fracture_type, patient_id)
#     #     shutil.copytree(source_path, target_path)
    
#     for patient_id in test_patients:
#         source_path = os.path.join(source_folder, patient_id)
#         target_path = os.path.join(target_dataset_folder, 'test', fracture_type, patient_id)
#         shutil.copytree(source_path, target_path)

# print("Dataset split completed.")


FileExistsError: [Errno 17] File exists: '/local/data1/honzh073/data/hospital_43_5fold/train/AFF/patient_RCQOXVJTTC'

In [5]:
import os

def count_number(path_folder):
    # AFF和NFF文件夹路径
    aff_folder = os.path.join(path_folder, 'AFF')
    nff_folder = os.path.join(path_folder, 'NFF')

    # 统计AFF和NFF文件夹中的图片数量
    aff_image_count = sum(len(files) for _, _, files in os.walk(aff_folder))
    nff_image_count = sum(len(files) for _, _, files in os.walk(nff_folder))
    
    return aff_image_count, nff_image_count

print('total:')
path_folder = '/local/data1/honzh073/data/hospital_43'
total_aff_num, total_nff_num = count_number(path_folder)
print('aff number:', total_aff_num, '; nff number:', total_nff_num)

print('train:')
path_folder = '/local/data1/honzh073/data/hospital_43_5fold/train'
aff_num, nff_num = count_number(path_folder)
print('aff number:', aff_num, '; nff number:', nff_num)
print('%:', aff_num / total_aff_num , '; %:', nff_num / total_nff_num)

print('test:')

path_folder = '/local/data1/honzh073/data/hospital_43_5fold/test'
aff_num, nff_num = count_number(path_folder)
print('aff number:', aff_num, '; nff number:', nff_num)
print('%:', aff_num / total_aff_num , '; %:', nff_num / total_nff_num)


# ---------------------------------------------------------------------------------

# aff and nff numbers in training dataset
train_aff_count = sum(1 for _, label in train_dataset if label == 0)  # 0 AFF
train_nff_count = sum(1 for _, label in train_dataset if label == 1)  # 1 NFF

# ---------------------------------------------------------------------------------
# Calculate class weights
n_aff_train = train_aff_count
n_nff_train = train_nff_count

class_weight_aff = 1 / (2 * (n_aff_train / (n_aff_train + n_nff_train)))
class_weight_nff = 1 / (2 * (n_nff_train / (n_aff_train + n_nff_train)))

print(f"Class Weight for AFF (0): {class_weight_aff:.4f}")
print(f"Class Weight for NFF (1): {class_weight_nff:.4f}")



total:
aff number: 75 ; nff number: 170
train:
aff number: 50 ; nff number: 116
%: 0.6666666666666666 ; %: 0.6823529411764706
test:
aff number: 25 ; nff number: 54
%: 0.3333333333333333 ; %: 0.3176470588235294
Class Weight for AFF (0): 1.6600
Class Weight for NFF (1): 0.7155


In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import random
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
from torch.optim import lr_scheduler
import torchvision.models as models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold


torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

# ----------------------------------------------------------------------------------------------
# Augmentation
# Define data augmentation transforms
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(30),  # Randomly rotate the image up to 30 degrees
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust color
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
    # transforms.RandomPerspective(),  # Random perspective transformation
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random resized crop
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image to RGB
])

# Define the data loaders
train_dataset = ImageFolder(root='/local/data1/honzh073/data/hospital_43_5fold/train', transform=transform)
test_dataset = ImageFolder(root='/local/data1/honzh073/data/hospital_43_5fold/test', transform=transform)


# batch_size = 16

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# ----------------------------------------------------------------------------------------------
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    device = torch.device("cuda:0")  # Use GPU 0
else:
    print("CUDA is not available.")
    device = torch.device("cpu")
# ----------------------------------------------------------------------------------------------
print('done!')


CUDA is available!
done!


In [10]:
from torchvision.models import resnet152, ResNet152_Weights

# Define K-fold cross-validator with 5 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Settings
lr = 0.0001
step_size = 10
gamma = 0.1

# Class weights
class_weights = [class_weight_aff, class_weight_nff]
class_weights = torch.Tensor(class_weights).to(device)

# ResNet152
model = models.resnet152(pretrained=True)
model = model.to(device)

# Update the output layer
model.fc = nn.Linear(2048, 2)  # output layer classes number = dataset classes number
model = model.to(device)

# Load trained model's weights
checkpoint = torch.load('/local/data1/honzh073/saved_model/fracatlas.pth')
model.load_state_dict(checkpoint)  # Load the model's state_dict directly


# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


# 存储训练和验证的损失和准确度
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
test_accuracies = []


# Loop through the folds
for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset)):
    print(f'Fold {fold + 1}')
    
    # Create data loaders for this fold
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    
    train_loader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(train_dataset, batch_size=16, sampler=val_sampler)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True)

    # Training loop
    for epoch in range(10):  # adjust the number of epochs as needed
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        train_losses.append(train_loss / total_train)
        train_accuracy = 100 * correct_train / total_train
        train_accuracies.append(train_accuracy)
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        val_losses.append(val_loss / total_val)
        val_accuracy = 100 * correct_val / total_val
        val_accuracies.append(val_accuracy)
        
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss / total_train:.4f}, Train Acc: {train_accuracy:.2f}%, Validation Loss: {val_loss / total_val:.4f}, Validation Acc: {val_accuracy:.2f}%')

    # Evaluate the model on the test set after each fold
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    test_accuracy = 100 * test_correct / test_total
    test_accuracies.append(test_accuracy)

    print(f'Test Accuracy (Fold {fold + 1}): {test_accuracy:.2f}%')

# 打印所有折的平均准确度
print(f'Average Test Accuracy: {np.mean(test_accuracies):.2f}%')

/local/data1/honzh073/anaconda3/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/local/data1/honzh073/anaconda3/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1
Epoch 1, Train Loss: 1.1440, Train Acc: 58.39%, Validation Loss: 0.9590, Validation Acc: 52.94%
Epoch 2, Train Loss: 0.7752, Train Acc: 64.43%, Validation Loss: 0.6068, Validation Acc: 70.59%
Epoch 3, Train Loss: 0.6641, Train Acc: 70.47%, Validation Loss: 0.4458, Validation Acc: 82.35%
Epoch 4, Train Loss: 0.5269, Train Acc: 78.52%, Validation Loss: 0.5579, Validation Acc: 70.59%
Epoch 5, Train Loss: 0.4550, Train Acc: 82.55%, Validation Loss: 0.6313, Validation Acc: 58.82%
Epoch 6, Train Loss: 0.3608, Train Acc: 91.28%, Validation Loss: 0.5130, Validation Acc: 76.47%
Epoch 7, Train Loss: 0.3376, Train Acc: 91.95%, Validation Loss: 0.4896, Validation Acc: 88.24%
Epoch 8, Train Loss: 0.2888, Train Acc: 91.95%, Validation Loss: 0.3598, Validation Acc: 88.24%
Epoch 9, Train Loss: 0.2828, Train Acc: 91.95%, Validation Loss: 0.3525, Validation Acc: 82.35%
Epoch 10, Train Loss: 0.1823, Train Acc: 95.30%, Validation Loss: 0.4030, Validation Acc: 82.35%
Test Accuracy (Fold 1): 79.75%
F